# LLM Inference Notebook

This notebook contains the scripts for LLM inference using Ollama or other LLM services.
Sample SPL: 
* Using local Ollama LLM:
* * | makeresults | eval text="Answer the question and state your reasoning." | fit MLTKContainer algo=llm_rag_ollama_text_processing model_name="llama3.2" prompt="Who are you?" text into app:llm_rag_ollama_text_processing as LLM
* Using other LLM service:
* * | makeresults | eval text="Answer the question and state your reasoning." | fit MLTKContainer algo=llm_rag_ollama_text_processing llm_service=azure_openai prompt="Who are you?" text into app:llm_rag_ollama_text_processing as LLM 

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import requests
import time
from llama_index.core.llms import ChatMessage
from app.model.llm_utils import create_llm, create_embedding_model
# ...
# global constants
ollama_url = "http://ollama:11434"
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.22.1
pandas version: 2.0.3


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

In [4]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("llm_rag_ollama_text_processing")
print(df.describe())
print(param)

## Stage 2 - create and initialize a model

In [ ]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    model['hyperparameter'] = 42.0
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(init(df,param))

## Stage 3 - fit the model

In [ ]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

## Stage 4 - apply the model

In [12]:
# from fit command, we will pass parameters model and prompt.
# sample prompt: You will examine if the email content given by the user is phishing. 
#                Only output **Phishing** if the content is phishing. 
#                Only output **Legit** if the email is legitimate. Do not give extra information.
def apply(model,df,param):
    try:
        X = df["text"].values.tolist()
    except:
        cols={'Result': ["ERROR: Please make sure you have a field in the search result named \'text\'"], 'Duration': ["ERROR"]}
        returns=pd.DataFrame(data=cols)
        return returns

    try:
        prompt = param['options']['params']['prompt'].strip("\"")
    except:
        cols={'Result': ["ERROR: Please make sure you set the parameter \'prompt\'"], 'Duration': ["ERROR"]}
        returns=pd.DataFrame(data=cols)
        return returns

    try:
        service = param['options']['params']['llm_service'].strip("\"")
        print(f"Using {service} LLM service.")
    except:
        service = "ollama"
        print("Using default Ollama LLM service.")
        
    try:
        model_name = param['options']['params']['model_name'].strip("\"")
    except:
        model_name = None
        print("No model name specified")

    llm, m = create_llm(service=service, model=model_name)

    if llm is None:
        cols={'Result': [m], 'Duration': ["ERROR"]}
        returns=pd.DataFrame(data=cols)
        return returns

    outputs_label = []
    outputs_duration = []

    
    for i in range(len(X)):
        messages = [
            ChatMessage(role="user", content=X[i]),
            ChatMessage(role="user", content=param['options']['params']['prompt'].strip("\"")),
        ]
        start_time = time.time()

        try:
            response = llm.chat(messages)
            end_time = time.time()
            outputs_label.append(response)
            duration = round(end_time - start_time,2)
            duration = str(duration) + " s"
            outputs_duration.append(duration)
        except Exception as e:
            outputs_label.append(f"ERROR at LLM generation: {e}")
            outputs_duration.append("ERROR")
        
    cols={'Result': outputs_label, 'Duration': outputs_duration}
    returns=pd.DataFrame(data=cols)
    return returns

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [21]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

def compute(model,df,param):
    try:
        X = df["text"].values.tolist()
    except:
        cols={'Result': ["ERROR: Please make sure you have a field in the search result named \'text\'"], 'Duration': ["ERROR"]}
        returns=pd.DataFrame(data=cols)
        return returns

    try:
        prompt = param['options']['params']['prompt'].strip("\"")
    except:
        cols={'Result': ["ERROR: Please make sure you set the parameter \'prompt\'"], 'Duration': ["ERROR"]}
        returns=pd.DataFrame(data=cols)
        return returns

    try:
        service = param['options']['params']['llm_service'].strip("\"")
        print(f"Using {service} LLM service.")
    except:
        service = "ollama"
        print("Using default Ollama LLM service.")
        
    try:
        model_name = param['options']['params']['model_name'].strip("\"")
    except:
        model_name = None
        print("No model name specified")

    llm, m = create_llm(service=service, model=model_name)

    if llm is None:
        cols={'Result': [m], 'Duration': ["ERROR"]}
        returns=pd.DataFrame(data=cols)
        return returns

    outputs_label = []
    outputs_duration = []

    
    for i in range(len(X)):
        messages = [
            ChatMessage(role="user", content=X[i]),
            ChatMessage(role="user", content=param['options']['params']['prompt'].strip("\"")),
        ]
        start_time = time.time()

        try:
            response = llm.chat(messages)
            end_time = time.time()
            outputs_label.append(response)
            duration = round(end_time - start_time,2)
            duration = str(duration) + " s"
            outputs_duration.append(duration)
        except Exception as e:
            outputs_label.append(f"ERROR at LLM generation: {e}")
            outputs_duration.append("ERROR")
        
    cols={'Result': outputs_label, 'Duration': outputs_duration}
    returns=pd.DataFrame(data=cols)
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code